# Issue Classification Example

In this example we perform a logical regression on a dataset of github issues to predict the labels on newly entered tickets.

In a second step we try to find issues that deal with similar problems.

In [1]:
# required libraries
from github import Github
import os
import pandas as pd

In [12]:

# Data Preparation 

## retrieve the issues from github
access_token = USER = os.getenv('GH_API_ACCESS')
token = Github(access_token)
repo = token.get_repo('quarkusio/quarkus')
issues = repo.get_issues(state='open')
print("Number of issues in total: ", issues.totalCount)

## turn the data into a pandas dataframe
issues_list = []    
for issue in issues.get_page(0): # TODO: remove pagination later on
    print(issue.labels[0].name)
    issues_list.append([issue.number, issue.title, issue.body, issue.labels,  issue.state])
issues_df = pd.DataFrame(issues_list, columns=['number', 'title', 'body', 'labels', 'state'])
issues_df.head()


Number of issues in total:  2305
area/core
area/dependencies
area/documentation
area/platform
area/kubernetes
area/dependencies
area/arc
kind/bug
area/arc
area/documentation
area/kubernetes
area/gradle
kind/enhancement


IndexError: list index out of range